In [8]:
import numpy as np
import torch
from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据

from torch import nn
from torch.autograd import Variable

In [9]:
train_set = mnist.MNIST('./data', train=True, download=True)
test_set = mnist.MNIST('./data', train=False, download=True)

In [10]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    # x = x.reshape((-1,)) # 拉平
    x = torch.from_numpy(x)
    return x

In [11]:
train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True) # 重新载入数据集，申明定义的数据变换
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)
print(train_set[0][0].size())

torch.Size([28, 28])


In [5]:
from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [6]:
net = nn.Sequential(
    nn.Linear(784, 400),
    nn.ReLU(),
    nn.Linear(400, 200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1

In [ ]:
# 开始训练
losses = []
acces = []
eval_losses = []
eval_acces = []

for e in range(20):
    train_loss = 0
    train_acc = 0
    net.train()
    for im, label in train_data:
        im = Variable(im)
        label = Variable(label)
        # 前向传播
        out = net(im)
        loss = criterion(out, label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 记录误差
        train_loss += loss.item()
        # 计算分类的准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
        
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    # 在测试集上检验效果
    eval_loss = 0
    eval_acc = 0
    net.eval() # 将模型改为预测模式
    for im, label in test_data:
        im = Variable(im)
        label = Variable(label)
        out = net(im)
        loss = criterion(out, label)
        # 记录误差
        eval_loss += loss.item()
        # 记录准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
        
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(e, train_loss / len(train_data), train_acc / len(train_data), 
                     eval_loss / len(test_data), eval_acc / len(test_data)))

epoch: 0, Train Loss: 0.516336, Train Acc: 0.836487, Eval Loss: 0.271753, Eval Acc: 0.915447
epoch: 1, Train Loss: 0.167086, Train Acc: 0.947844, Eval Loss: 0.165379, Eval Acc: 0.947488
epoch: 2, Train Loss: 0.116513, Train Acc: 0.964486, Eval Loss: 0.105531, Eval Acc: 0.966377
epoch: 3, Train Loss: 0.093548, Train Acc: 0.970349, Eval Loss: 0.110521, Eval Acc: 0.966475
epoch: 4, Train Loss: 0.074893, Train Acc: 0.976013, Eval Loss: 0.087427, Eval Acc: 0.973695
epoch: 5, Train Loss: 0.067338, Train Acc: 0.978928, Eval Loss: 0.069662, Eval Acc: 0.978145
epoch: 6, Train Loss: 0.054614, Train Acc: 0.982909, Eval Loss: 0.076002, Eval Acc: 0.975771
epoch: 7, Train Loss: 0.046299, Train Acc: 0.985108, Eval Loss: 0.085380, Eval Acc: 0.976167
epoch: 8, Train Loss: 0.039111, Train Acc: 0.986674, Eval Loss: 0.079879, Eval Acc: 0.975178
epoch: 9, Train Loss: 0.033725, Train Acc: 0.989072, Eval Loss: 0.059630, Eval Acc: 0.981606
epoch: 10, Train Loss: 0.029467, Train Acc: 0.989922, Eval Loss: 0.075